In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf

# 1. Загрузка и подготовка данных
print("1. Загрузка и подготовка данных...")
data = pd.read_csv("CupIT_Sber_data.csv", delimiter=";")

# Преобразование типов и очистка
data['service_amount_net'] = pd.to_numeric(data['service_amount_net'], errors='coerce')
data['service_date'] = pd.to_datetime(data['service_date'])
data = data.dropna(subset=['service_date', 'service_amount_net'])

# 2. Создание временных лагов и фичей
print("\n2. Создание временных лагов и признаков...")

# Агрегация по месяцам с сохранением всех колонок
monthly_data = data.resample('M', on='service_date').agg({
    'service_amount_net': 'sum',
    'service_document_id': 'count',
    'patient_id': 'nunique',
    'service_code': 'nunique',
    'is_hospital': 'mean'
}).rename(columns={
    'service_document_id': 'transactions_count',
    'patient_id': 'unique_patients',
    'service_code': 'unique_services'
})

# Добавим скользящие статистики
monthly_data['rolling_mean_3'] = monthly_data['service_amount_net'].rolling(window=3).mean()
monthly_data['rolling_std_3'] = monthly_data['service_amount_net'].rolling(window=3).std()
monthly_data['rolling_mean_12'] = monthly_data['service_amount_net'].rolling(window=12).mean()
monthly_data['rolling_std_12'] = monthly_data['service_amount_net'].rolling(window=12).std()

# Добавим разности и процентные изменения
monthly_data['diff_1'] = monthly_data['service_amount_net'].diff(1)
monthly_data['pct_change_1'] = monthly_data['service_amount_net'].pct_change(1)
monthly_data['diff_12'] = monthly_data['service_amount_net'].diff(12)
monthly_data['pct_change_12'] = monthly_data['service_amount_net'].pct_change(12)

# Лаги для целевой переменной
for lag in [1, 2, 3, 6, 9, 12, 24]:
    monthly_data[f'lag_{lag}'] = monthly_data['service_amount_net'].shift(lag)

# Дополнительные временные фичи
monthly_data['month'] = monthly_data.index.month
monthly_data['year'] = monthly_data.index.year
monthly_data['quarter'] = monthly_data.index.quarter
monthly_data['days_in_month'] = monthly_data.index.days_in_month
monthly_data['is_month_start'] = monthly_data.index.is_month_start.astype(int)
monthly_data['is_month_end'] = monthly_data.index.is_month_end.astype(int)

# Добавим тригонометрические фичи для сезонности
monthly_data['month_sin'] = np.sin(2 * np.pi * monthly_data['month']/12)
monthly_data['month_cos'] = np.cos(2 * np.pi * monthly_data['month']/12)

# Добавим экспоненциальное сглаживание
monthly_data['ewma_3'] = monthly_data['service_amount_net'].ewm(span=3, adjust=False).mean()
monthly_data['ewma_12'] = monthly_data['service_amount_net'].ewm(span=12, adjust=False).mean()

# Добавим фичи из декомпозиции временного ряда
try:
    decomposition = seasonal_decompose(monthly_data['service_amount_net'].dropna(), model='additive', period=12)
    monthly_data['trend'] = decomposition.trend
    monthly_data['seasonal'] = decomposition.seasonal
    monthly_data['residual'] = decomposition.resid
except:
    print("Не удалось выполнить декомпозицию временного ряда")

# Добавим автокорреляционные фичи
try:
    acf_values = acf(monthly_data['service_amount_net'].dropna(), nlags=12)
    for lag in range(1, 7):
        monthly_data[f'acf_{lag}'] = acf_values[lag]
except:
    print("Не удалось вычислить автокорреляцию")

# Удаление строк с пропусками
monthly_data = monthly_data.dropna()

# 3. Подготовка train/test
print("\n3. Разделение данных на train/test...")
train_size = int(len(monthly_data) * 0.8)
train = monthly_data.iloc[:train_size]
test = monthly_data.iloc[train_size:]

X_train, y_train = train.drop('service_amount_net', axis=1), train['service_amount_net']
X_test, y_test = test.drop('service_amount_net', axis=1), test['service_amount_net']

# 4. Оптимизация гиперпараметров с Optuna
print("\n4. Оптимизация гиперпараметров LightGBM...")

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'verbosity': -1,
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
    }
    
    # Кросс-валидация временного ряда
    tscv = TimeSeriesSplit(n_splits=3)
    scores = []
    
    for train_idx, val_idx in tscv.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold)
        
        model = lgb.train(params,
                         train_data,
                         valid_sets=[val_data],
                         callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
        
        preds = model.predict(X_val_fold)
        score = mean_absolute_error(y_val_fold, preds)
        scores.append(score)
    
    return np.mean(scores)

study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(objective, n_trials=50, show_progress_bar=True)

# 5. Обучение лучшей модели
print("\n5. Обучение лучшей модели LightGBM...")
best_params = study.best_params
best_params['verbosity'] = -1

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

model = lgb.train(best_params,
                 train_data,
                 valid_sets=[test_data],
                 callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True)])

# 6. Прогнозирование и оценка
print("\n6. Прогнозирование и оценка модели...")
test['forecast'] = model.predict(X_test)

# Расчет доверительного интервала
n_bootstrap = 100
preds = []
for _ in tqdm(range(n_bootstrap), desc="Bootstrap"):
    sample_idx = np.random.choice(len(X_test), len(X_test), replace=True)
    preds.append(model.predict(X_test.iloc[sample_idx]))
    
preds = np.array(preds)
test['lower_ci'] = np.percentile(preds, 2.5, axis=0)
test['upper_ci'] = np.percentile(preds, 97.5, axis=0)

# 7. Визуализация результатов
plt.figure(figsize=(14, 7))
plt.plot(train.index, train['service_amount_net'], label='Обучающая выборка', linewidth=2)
plt.plot(test.index, test['service_amount_net'], label='Фактические значения', 
         color='green', linewidth=2)
plt.plot(test.index, test['forecast'], label='Прогноз', 
         color='red', linestyle='--', linewidth=2)
plt.fill_between(test.index,
                test['lower_ci'],
                test['upper_ci'], color='pink', alpha=0.3)
plt.title('Сравнение прогноза с реальными значениями (LightGBM)', fontsize=14)
plt.xlabel('Дата', fontsize=12)
plt.ylabel('Сумма выплат', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# 8. Расчет метрик
mae = mean_absolute_error(test['service_amount_net'], test['forecast'])
rmse = np.sqrt(mean_squared_error(test['service_amount_net'], test['forecast']))
mape = np.mean(np.abs((test['service_amount_net'] - test['forecast']) / test['service_amount_net'])) * 100

print("\nМетрики качества на тестовой выборке:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# 9. Прогноз на будущие периоды с правильным обновлением лагов
print("\n9. Прогноз на будущие периоды с последовательным обновлением лагов...")

def create_rolling_forecast(model, last_date, last_values, steps):
    # Создаем копию последних значений для лагов
    history = last_values.copy()
    future_dates = pd.date_range(start=last_date, periods=steps+1, freq='M')[1:]
    forecasts = []
    lower_cis = []
    upper_cis = []
    
    # Создаем шаблон строки с медианными значениями для всех признаков
    template_row = monthly_data.median().to_dict()
    
    for i in range(steps):
        # Создаем DataFrame для прогноза текущего месяца
        X_pred = pd.DataFrame(template_row, index=[future_dates[i]])
        
        # Обновляем временные признаки
        X_pred['month'] = future_dates[i].month
        X_pred['year'] = future_dates[i].year
        X_pred['quarter'] = (future_dates[i].month-1)//3 + 1
        X_pred['days_in_month'] = future_dates[i].days_in_month
        X_pred['is_month_start'] = (future_dates[i].is_month_start).astype(int)
        X_pred['is_month_end'] = (future_dates[i].is_month_end).astype(int)
        
        # Обновляем тригонометрические фичи
        X_pred['month_sin'] = np.sin(2 * np.pi * X_pred['month']/12)
        X_pred['month_cos'] = np.cos(2 * np.pi * X_pred['month']/12)
        
        # Обновляем лаги
        for lag in [1, 2, 3, 6, 9, 12, 24]:
            if len(history) >= lag:
                X_pred[f'lag_{lag}'] = history.iloc[-lag]
            else:
                X_pred[f'lag_{lag}'] = history.mean()
        
        # Обновляем скользящие статистики
        if len(history) >= 3:
            X_pred['rolling_mean_3'] = history.iloc[-3:].mean()
            X_pred['rolling_std_3'] = history.iloc[-3:].std()
        if len(history) >= 12:
            X_pred['rolling_mean_12'] = history.iloc[-12:].mean()
            X_pred['rolling_std_12'] = history.iloc[-12:].std()
        
        # Обновляем разности
        if len(history) >= 1:
            X_pred['diff_1'] = history.iloc[-1] - history.iloc[-2] if len(history) >= 2 else 0
            X_pred['pct_change_1'] = (history.iloc[-1] - history.iloc[-2])/history.iloc[-2] if len(history) >= 2 and history.iloc[-2] != 0 else 0
        if len(history) >= 12:
            X_pred['diff_12'] = history.iloc[-1] - history.iloc[-13] if len(history) >= 13 else 0
            X_pred['pct_change_12'] = (history.iloc[-1] - history.iloc[-13])/history.iloc[-13] if len(history) >= 13 and history.iloc[-13] != 0 else 0
        
        # Обновляем экспоненциальное сглаживание
        if len(history) >= 1:
            X_pred['ewma_3'] = history.ewm(span=3, adjust=False).mean().iloc[-1]
            X_pred['ewma_12'] = history.ewm(span=12, adjust=False).mean().iloc[-1]
        
        # Убедимся, что порядок столбцов совпадает с обучающими данными
        X_pred = X_pred[X_train.columns]
        
        # Прогнозируем текущий месяц
        current_forecast = model.predict(X_pred)[0]
        forecasts.append(current_forecast)
        
        # Для доверительных интервалов используем историческую волатильность
        historical_std = history.pct_change().std()
        if pd.isna(historical_std) or historical_std == 0:
            historical_std = 0.1  # значение по умолчанию
            
        lower_cis.append(current_forecast * (1 - 1.96 * historical_std))
        upper_cis.append(current_forecast * (1 + 1.96 * historical_std))
        
        # Обновляем историю для следующего шага
        history = pd.concat([history, pd.Series([current_forecast], index=[future_dates[i]])])
    
    # Создаем DataFrame с результатами
    result = pd.DataFrame({
        'date': future_dates,
        'forecast': forecasts,
        'lower_ci': lower_cis,
        'upper_ci': upper_cis
    }).set_index('date')
    
    return result

# Прогноз на 12 месяцев
last_date = monthly_data.index[-1]
last_values = monthly_data['service_amount_net']
future_12m = create_rolling_forecast(model, last_date, last_values, 12)

# Визуализация прогноза на 12 месяцев
plt.figure(figsize=(14, 7))
plt.plot(monthly_data.index[-24:], monthly_data['service_amount_net'][-24:], 
         label='Исторические данные', linewidth=2)
plt.plot(future_12m.index, future_12m['forecast'], 
         label='Прогноз на 12 месяцев', color='red', marker='o', linewidth=2)
plt.fill_between(future_12m.index,
                future_12m['lower_ci'],
                future_12m['upper_ci'], color='pink', alpha=0.3)
plt.title('Прогноз месячных выплат на 12 месяцев вперед', fontsize=14)
plt.xlabel('Дата', fontsize=12)
plt.ylabel('Сумма выплат', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("\nПрогноз на 12 месяцев:")
print(future_12m)

# 10. Важность признаков
plt.figure(figsize=(12, 8))
lgb.plot_importance(model, importance_type='gain', max_num_features=20, figsize=(12, 8))
plt.title('Важность признаков (LightGBM) по gain', fontsize=14)
plt.tight_layout()
plt.show()

# 11. Анализ остатков
test['residuals'] = test['service_amount_net'] - test['forecast']

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.scatter(test['forecast'], test['residuals'], alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Прогноз')
plt.ylabel('Остатки')
plt.title('Остатки vs Прогноз')

plt.subplot(1, 2, 2)
test['residuals'].plot(kind='hist', bins=30)
plt.title('Распределение остатков')
plt.tight_layout()
plt.show()

# 12. Сохранение результатов
forecast_df = future_12m.reset_index()
forecast_df.to_csv('improved_payments_forecast.csv', index=False)
print("\nПрогноз сохранен в improved_payments_forecast.csv")

1. Загрузка и подготовка данных...

2. Создание временных лагов и признаков...


[I 2025-03-27 12:33:23,337] A new study created in memory with name: no-name-bdffdee9-e3a3-4fad-8d2e-f1e5641b7f34



3. Разделение данных на train/test...

4. Оптимизация гиперпараметров LightGBM...


  0%|          | 0/50 [00:00<?, ?it/s]

[W 2025-03-27 12:33:23,341] Trial 0 failed with parameters: {'n_estimators': 615, 'max_depth': 4, 'learning_rate': 0.06754598280950812, 'subsample': 0.7089802720538816, 'colsample_bytree': 0.7121775086659876, 'num_leaves': 113, 'min_child_samples': 44, 'reg_alpha': 8.288502468754423, 'reg_lambda': 4.753909671652027, 'feature_fraction': 0.5360502893383087, 'bagging_fraction': 0.7930748765245053, 'bagging_freq': 6} because of the following error: ValueError('Cannot have number of folds=4 greater than the number of samples=0.').
Traceback (most recent call last):
  File "C:\Users\Mi\AppData\Roaming\Python\Python312\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Mi\AppData\Local\Temp\ipykernel_18208\4061845926.py", line 126, in objective
    for train_idx, val_idx in tscv.split(X_train):
  File "c:\Python312\Lib\site-packages\sklearn\model_selection\_split.py", line 1276, in _split
    r

ValueError: Cannot have number of folds=4 greater than the number of samples=0.